## Test accuracy on 20 CIFAR-100 classes using ResNet50V2

### Import all the necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

2025-05-26 23:17:41.564828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748279861.585876   29275 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748279861.591425   29275 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748279861.608317   29275 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748279861.608355   29275 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748279861.608359   29275 computation_placer.cc:177] computation placer alr

###  Load CIFAR-100 fine labels (100-class)

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [3]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [4]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 224x224

In [5]:
x_train_20 = tf.image.resize(x_train_20, (224, 224)).numpy()
x_test_20 = tf.image.resize(x_test_20, (224, 224)).numpy()

I0000 00:00:1748279865.918115   29275 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 173 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-05-26 23:17:45.919355: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
I0000 00:00:1748279865.949558   29275 cuda_executor.cc:479] failed to allocate 173.00MiB (181403648 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2025-05-26 23:17:45.962749: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2025-05-26 23:17:45.986119: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6021120000 exceeds 10% of free system memory.
2025-05-26 23:18:21.499783: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1204224000 exceeds 10%

### Preprocess

In [6]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [7]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load ResNet50V2 base model

In [8]:
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using ResNet50V2:", acc)

2025-05-26 23:18:41.879317: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5419008000 exceeds 10% of free system memory.
2025-05-26 23:19:34.407462: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.05GiB (rounded to 5419008000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-26 23:19:34.411907: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-26 23:19:34.412908: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 71, Chunks in use: 71. 17.8KiB allocated for chunks. 17.8KiB in use in bin. 7.5KiB client-requested in use in bin.
2025-05-26 23:19:34.413197: I external/local_xla/xla/tsl/framework/bfc_allocator.

KeyboardInterrupt: 